In [ ]:
import os
import sys
from datetime import datetime


from config import MAILSLURP_API
import mailslurp_client
from email.utils import formatdate
from email.message import EmailMessage
import email.header
from mailslurp_client import ApiException, Email, EmailPreview



print(mailslurp_client.__version__)


# Utils

In [ ]:
def wrap_email(email_mailslurp: mailslurp_client.Email) -> EmailMessage:
    #raise NotImplementedError(f"wrap_email method not yet implemented for MailSlurp(because ran out of API credits!!!)")
    msg = EmailMessage()
    msg['Subject'] = email_mailslurp.subject
    msg['From'] = email_mailslurp._from
    msg['To'] = email_mailslurp._to
    msg['Date'] = formatdate(email_mailslurp.created_at.timestamp())
    msg.set_content(email_mailslurp.body)
    return msg  # returns the Email Object


In [ ]:
import re 
import logging
log = logging.getLogger(__name__)

def validate_email_addr(email_addr):
    """ Ensure that emaill address conforms to valid structure and return if True """

    EMAIL_REGEX = re.compile(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', re.I)

    log.debug(f"Validationg: {email_addr}")        
    if(re.fullmatch(EMAIL_REGEX, email_addr)):
        return email_addr
    else:
        raise ValueError("Invalid Email...")

In [ ]:
validate_email_addr('as@asfg.com')

In [ ]:
test_str = ' asdomte!)(*@!ststringemail.com'
test_str.lstrip().split("@")[0]

# Setup


## Configuration

In [62]:
# set up required MailSlurp Configuration
configuration = mailslurp_client.Configuration()
configuration.api_key['x-api-key'] = MAILSLURP_API

# create an API client
api_client = mailslurp_client.ApiClient(configuration)

# create an InboxControllerApi  # See: https://docs.mailslurp.com/ruby/docs/InboxControllerApi/
inbox_controller = mailslurp_client.InboxControllerApi(api_client)
waitfor_controller = mailslurp_client.WaitForControllerApi(api_client)
email_controller = mailslurp_client.EmailControllerApi(api_client)


In [66]:
email_controller.get_latest_email_in_inbox1(inbox_id='dfcb3b3f-aa43-43a4-a29e-8844b0352af1')

{'_from': 'ajanderson1@gmail.com',
 'analysis': {'dkim_verdict': 'PASS',
              'dmarc_verdict': 'PASS',
              'spam_verdict': 'PASS',
              'spf_verdict': 'PASS',
              'virus_verdict': 'PASS'},
 'attachments': [],
 'bcc': [],
 'body': '<div dir="ltr">testemail body 1<div><br></div></div>\r\n',
 'body_excerpt': '<div dir="ltr">testemail body 1<div><br></div></div>\r\n',
 'body_md5_hash': '2B121C22569855B6DEE5BDA5270A57E5',
 'cc': [],
 'charset': 'UTF-8',
 'created_at': datetime.datetime(2022, 7, 10, 14, 11, 42, 451000, tzinfo=tzutc()),
 'headers': {'Authentication-Results': 'amazonses.com;\r\n'
                                       ' spf=pass (spfCheck: domain of '
                                       '_spf.google.com designates '
                                       '209.85.210.175 as permitted sender) '
                                       'client-ip=209.85.210.175; '
                                       'envelope-from=ajanderson1@gmail.com; '

# Inbox Fuctions

## Show all inboxes

In [ ]:
inbox_controller.get_all_inboxes()
# inbox_controller.get_all_inboxes_with_http_info()

## Delete all inboxes
!! CAREFUL

## Number of Inboxes

In [41]:
inbox_controller.get_inbox_count()

{'total_elements': 7}

## Flush expired Inboxes

In [ ]:
inbox_controller.flush_expired()

## Set Inbox to first inbox

In [ ]:
this_inbox = inbox_controller.get_all_inboxes().content[0]
print(f"Inbox set to \nemail address: {inbox_controller.get_all_inboxes().content[0].email_address}")
print(this_inbox.id)

# print(f"Inbox set to \nID: {this_inbox.id}\nemail address:{this_inbox.email_address}"): {this_inbox.id}\nemail address:{this_inbox.email_address}")

In [ ]:
list_of_avail_inboxes: list[mailslurp_client.models.inbox_dto.InboxDto] = inbox_controller.get_all_inboxes().content
type(list_of_avail_inboxes[0])

## get back an inbox from an id or an emaill

In [42]:
an_id = 'dfcb3b3f-aa43-43a4-a29e-8844b0352af1'
# an_emaill_address = 
type(inbox_controller.get_inbox(an_id))

mailslurp_client.models.inbox_dto.InboxDto

## Set Inbox to specific inbox

In [ ]:
this_inbox_id = "aadaf10e-800e-4408-b982-685b13d6ca7a"

try:
    this_inbox = inbox_controller.get_inbox(this_inbox_id)
    print(f"Inbox set to \nID: {this_inbox.id}\nemail address: {this_inbox.email_address}")
except ApiException:
    print(f"ApiError: likely that the inbox_id: {this_inbox_id} is incorrect")


## Create  new inbox (with random domain)

In [ ]:
inbox_controller.create_inbox(use_domain_pool=True)

## Get emails (emailPreview) from inbox

In [61]:
all_emails = email_controller.get_latest_email()
# print((all_emails[0]).__dict__)  # NB: this retutned type mailslup_client.emailPreview
for email in all_emails:
    print(f"{email.id} // {email.subject} // {email._from}")

TypeError: 'Email' object is not iterable

In [58]:
this_inbox_id = "dfcb3b3f-aa43-43a4-a29e-8844b0352af1"
this_inbox = inbox_controller.get_inbox(this_inbox_id)

email = email_controller.get_latest_email_in_inbox1(this_inbox.id)
wrapped_email = wrap_email(email)
print(wrapped_email.as_string())

ApiException: (404)
Reason: 
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 10 Jul 2022 14:07:04 GMT', 'Content-Type': 'text/plain;charset=UTF-8', 'Content-Length': '426', 'Connection': 'keep-alive', 'X-XSS-Protection': '1; mode=block', 'Referrer-Policy': 'same-origin', 'Strict-Transport-Security': 'max-age=63072000; includeSubdomains; preload', 'X-Content-Type-Options': 'nosniff', 'Content-Security-Policy': "default-src https: 'self' app.mailslurp.com *.mailslurp.com *.mailslurp.link *.mailslurp.dev http://localhost:3000; object-src none; frame-ancestors 'self' app.mailslurp.com *.mailslurp.com *.mailslurp.link *.mailslurp.dev http://localhost:3000; img-src * data: 'unsafe-inline'; style-src * data: 'unsafe-inline';", 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers'})
HTTP response body: {"message":"No messages found for given user. Have you received an email yet? (GET null)","caseNumber":"CASE-NF-ddb0a374-8ec7-4d09-be8e-6a9e5baf0075","comment":"If

## Get email from emailID (from emailPreview)

In [ ]:
this_email = email_controller.get_email(all_emails[0].id)
this_email.body

# Emails

In [ ]:
# list all inboxes
with mailslurp_client.ApiClient(configuration) as api_client:
    inbox_controller = mailslurp_client.InboxControllerApi(api_client)
    inboxes = inbox_controller.get_all_inboxes(page=0)

    # pagination properties
    assert inboxes.total_pages > 0
    assert inboxes.total_elements > 0

    # view contents
    assert len(inboxes.content[0].id) > 0
    for this_inbox in inboxes.content:
        print (this_inbox.id)


In [ ]:
inbox = inboxes.content[0]
# print(inbox.__dict__)
print(help(inbox))